In [ ]:
import os, sys,pytz
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()

Put mother on follow up schedule and link to the child pid 272-1 on dashboard for -10


In [ ]:
from edc_visit_schedule.models import SubjectScheduleHistory

subject_identifier = 'B142-040991272-1'
child_subject_identifier ='B142-040991272-1-10'
schedule_name_mother = 'c_quarterly1_schedule1'
schedule_name_child = 'child_c_fu_schedule1'

subject_history_obj_child = SubjectScheduleHistory.objects.get(
subject_identifier=child_subject_identifier,
schedule_name = schedule_name_child
)

onschedule_model_cls_child= django_apps.get_model(subject_history_obj_child.onschedule_model)

onschedule_model_obj_child = onschedule_model_cls_child.objects.get(
    subject_identifier=child_subject_identifier,
    schedule_name=schedule_name_child,
    onschedule_datetime=subject_history_obj_child.onschedule_datetime,
)

onschedule_datetime = onschedule_model_obj_child.onschedule_datetime


In [ ]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from flourish_caregiver.models import MaternalVisit,OnScheduleCohortCFU
from flourish_child.models.onschedule import OnScheduleChildCohortCFU
import pytz
tz = pytz.timezone('Africa/Gaborone')

maternal_visits = MaternalVisit.objects.filter(subject_identifier=subject_identifier,visit_code='2003M')
for maternal_visit in maternal_visits:
    if maternal_visit.child_subject_identifier == child_subject_identifier:
        child_schedule = OnScheduleChildCohortCFU.objects.get(subject_identifier=child_subject_identifier)
        schedule_name= 'c_fu1_schedule1'

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortcfu',name=schedule_name
)
schedule.put_on_schedule(
    subject_identifier=subject_identifier,onschedule_datetime=onschedule_datetime,
    schedule_name=schedule_name,base_appt_datetime=onschedule_datetime
)
print("Schedule created for",schedule)

onschdule_mothers= OnScheduleCohortCFU.objects.filter(subject_identifier=subject_identifier)
for onschedule_mother in onschdule_mothers:
    if onschedule_mother.child_subject_identifier == '':
        onschedule_mother.child_subject_identifier =child_schedule.subject_identifier
        onschedule_mother.save()

        print("Linked with child")
